<a href="https://colab.research.google.com/github/YdoUcare-qm/Smart-Waste-Management/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pillow matplotlib torchvision


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os


In [4]:
# Define file paths
csv_path = '/content/drive/MyDrive/Smart Waste Management in Modern Cities/Train-Data.csv'
image_folder_path = '/content/drive/MyDrive/Smart Waste Management in Modern Cities/Image-Files/'

# Load CSV data
data = pd.read_csv(csv_path)

# Display the first few rows
data.head()


,image,query,label,startX,startY,endX,endY
0,398faec8-6799-11e5-8dc4-40f2e96c8ad8.jpg,city garbage,1.0,18.0,113.0,528.0,427.0
1,3adba2c8-6799-11e5-8dc4-40f2e96c8ad8.jpg,city garbage,NaN,NaN,NaN,NaN,NaN
2,46e3e5ee-6799-11e5-8dc4-40f2e96c8ad8.jpg,city garbage,1.0,188.0,183.0,553.0,401.0
3,47e1b9a8-6799-11e5-8dc4-40f2e96c8ad8.jpg,city garbage,NaN,NaN,NaN,NaN,NaN
4,495f1dca-6799-11e5-8dc4-40f2e96c8ad8.jpg,city garbage,1.0,270.0,306.0,369.0,340.0


In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset

# Define a simple CNN for classification
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 20)  # Adjust num_classes according to your dataset

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = SimpleCNN()


In [94]:
#pip install --upgrade torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1+cu121
    Uninstalling torchvision-0.18.1+cu121:
      Successfully uni

In [8]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a pre-trained Faster R-CNN model
model_detection = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Extract the number of input features for the classifier
in_features = model_detection.roi_heads.box_predictor.cls_score.in_features

# Define the number of classes (including background)
num_classes = 20  # Example: 2 (background + one object class)

# Replace the box predictor with a new one for our number of classes
model_detection.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_detection.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [9]:
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
import os

from torchvision.transforms import ToTensor

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform or ToTensor()  # Convert images to tensors by default

        # Remove rows with missing bounding box values
        self.annotations = self.annotations.dropna(subset=['startX', 'startY', 'endX', 'endY'])

        # Create a mapping from class names to integers
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.annotations['query'].unique())}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
      subfolder = self.annotations.iloc[idx, 1]  # query
      img_name = os.path.join(self.root_dir, subfolder, self.annotations.iloc[idx, 0])
      image = Image.open(img_name).convert("RGB")

      # Convert class name to integer label
      label = torch.tensor(self.class_to_idx[self.annotations.iloc[idx, 1]])

      # Extract and normalize bounding box coordinates
      bbox = torch.tensor([
          self.annotations.iloc[idx, 3],  # startX
          self.annotations.iloc[idx, 4],  # startY
          self.annotations.iloc[idx, 5],  # endX
          self.annotations.iloc[idx, 6]   # endY
      ], dtype=torch.float32)

      # Sanity checks
      if bbox[0] < 0 or bbox[1] < 0 or bbox[2] <= bbox[0] or bbox[3] <= bbox[1]:
          raise ValueError(f"Invalid bounding box: {bbox}")

      if label < 0 or label >= num_classes:
          raise ValueError(f"Invalid label: {label}")

      if self.transform:
          image = self.transform(image)

      return image, {'boxes': bbox.unsqueeze(0), 'labels': label.unsqueeze(0)}




In [10]:
# Define file paths
csv_path = '/content/drive/MyDrive/Smart Waste Management in Modern Cities/Train-Data.csv'
image_folder_path = '/content/drive/MyDrive/Smart Waste Management in Modern Cities/Image-Files/'

# Create an instance of the CustomDataset
dataset = CustomDataset(csv_file=csv_path, root_dir=image_folder_path, transform=None)


In [12]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


# Load a pre-trained Faster R-CNN model
model_detection = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Extract the number of input features for the classifier
in_features = model_detection.roi_heads.box_predictor.cls_score.in_features

# Define the number of classes (including background)
num_classes = 20  # Example: Adjust based on your dataset

# Replace the box predictor with a new one for our number of classes
model_detection.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_detection.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [13]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)

dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [19]:
for images, targets in dataloader:
    print("Images batch shape:", [img.shape for img in images])
    print("Targets batch:", targets)
    break

Images batch shape: [torch.Size([3, 423, 638]), torch.Size([3, 415, 634])]
Targets batch: [{'boxes': tensor([[138., 216., 244., 340.]]), 'labels': tensor([0])}, {'boxes': tensor([[  4., 199., 633., 411.]]), 'labels': tensor([1])}]


In [28]:
num_epochs = 2
optimizer = optim.Adam(model_detection.parameters(), lr=0.01)
# Training loop
for epoch in range(num_epochs):
    model_detection.train()
    running_loss = 0.0
    for images, targets in dataloader:
        images = [image.to(device) for image in images]  # images are now tensors
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        # Forward pass
        loss_dict = model_detection(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass and optimization
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    epoch_loss = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


Epoch [1/2], Loss: 53330726103741375634365132832768.0000
Epoch [2/2], Loss: 53522603369041736287522402598912.0000


In [36]:
# Define a different path
model_save_path = '/content/drive/MyDrive/faster_rcnn_model.pth'

# Save the model's state dictionary
torch.save(model_detection.state_dict(), model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to /content/drive/MyDrive/faster_rcnn_model.pth


In [21]:
from sklearn.metrics import precision_recall_fscore_support

def compute_f1_score(predictions, targets):
    pred_labels = [p['labels'].cpu().numpy() for p in predictions]
    true_labels = [t['labels'].cpu().numpy() for t in targets]

    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
    return f1


In [22]:
import torch

def compute_iou(pred_boxes, true_boxes):
    def box_area(boxes):
        x1, y1, x2, y2 = boxes
        return (x2 - x1) * (y2 - y1)

    def intersect(boxes1, boxes2):
        x1 = max(boxes1[0], boxes2[0])
        y1 = max(boxes1[1], boxes2[1])
        x2 = min(boxes1[2], boxes2[2])
        y2 = min(boxes1[3], boxes2[3])
        return max(x2 - x1, 0) * max(y2 - y1, 0)

    def iou(boxes1, boxes2):
        intersect_area = intersect(boxes1, boxes2)
        union_area = box_area(boxes1) + box_area(boxes2) - intersect_area
        return intersect_area / union_area if union_area > 0 else 0

    ious = [iou(p, t) for p, t in zip(pred_boxes, true_boxes)]
    return max(ious) if ious else 0
